In [1]:
import torch
import numpy as np

# tensors are specialized data structures that are very similar to arrays and matrices
# in pytorch they are used to encode the inputs, outputs and parameters.
# very similar to NP ndarray but can run on GPUS for specialized computing

In [12]:
# torch approach
data = [[1,2],[3,4]]
x_data = torch.tensor(data)

# numpy approach
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

# from another tensor (this just replaces the entires with 1s but retains the properties)

x_ones = torch.ones_like(x_data)

In [13]:
# shape is tuple of dimensions rows, columns 

shape = (2,3,)
tensor = torch.rand(shape)

# the attributes of a tensor are shape, dtype, device where it is stored

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([2, 3])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [15]:
# you can pass the computation device to GPU via cuda

if torch.cuda.is_available():
    tensor = tensor.to('cuda')
    print(f"Device tensor is stored on: {tensor.device}")
else:
    print("not available")

not available


In [ ]:
tensor[1,:] = -5
tensor_rand = torch.rand(2,3)

# concat to be rows (0) concat to be columns (1)
t1 = torch.cat([tensor,tensor_rand],dim = 1 )
t1 


tensor([[-5.0000, -5.0000, -5.0000,  0.2578,  0.1744,  0.3483],
        [-5.0000, -5.0000, -5.0000,  0.9094,  0.9282,  0.8958]])

In [ ]:
# multiplying is straightforward just * or tensor.mul
tensor * tensor_rand

# matrix multiplication is @
tensor @ torch.transpose(tensor_rand,0,1)

tensor([[ -3.9026, -13.6669],
        [ -3.9026, -13.6669]])

In [ ]:
# you can switch from tensors to numpy arrays if both are in the CPU 
t = torch.ones(2)
n = t.numpy()
# and you can still change n via t 
# the other direction is 
b = torch.from_numpy(n)

In [ ]:
# intro to torch.autograd 
# this is the automatic differentiation engine that powers nueral network training.
#Neural Networks are a collection of nested functions that are executed on some input data. 
# These functions are parameters (weights and biases), which in PyTorch are stored in tensors.
# Steps to Train a NN
# Forward Propagation: NN makes its best guess about the correct ouput. Runs the input through its functions to guess
# Backward Propagation: NN adjust its parameters proportionate to the error. Done by traversing form the ouput while collecting derivatives of the error with respect to parameters (gradients) de/dp
# Then optimizes the parameters using gradient descent

In [ ]:

import torch
from torchvision.models import resnet18, ResNet18_Weights
#pretrained resnet18 model 
model = resnet18(weights=ResNet18_Weights.DEFAULT)
# create random tensor 1 batch(image count) of 3 channels (columns) each of height and width 64 
data = torch.rand(1, 3, 64, 64)
# something to compare to i suppose
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/blakekell/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100.0%


In [ ]:
prediction = model(data) # forward propagation


In [31]:
loss = (prediction - labels).sum()
loss.backward() # back propagation

In [ ]:
# load the optimizer
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [ ]:
# .step() initiates gradient descent
optim.step()

In [ ]:
import torch 
a = torch.tensor([2.,3.], requires_grad=True)
b = torch.tensor([6.,4.], requires_grad=True)

Q = 3*a**3 - b**2

# requires_grad makes every operation be track as in mathematical operation 

In [36]:
Q.sum().backward()



In [37]:
# check if collected gradients are correct
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


In [ ]:
# Neural Networks
# constructed with the torch.nn package. nn depends on autograd to define models and differntiate them
# nn.module contains laters and a method forward(input) that retursn the output


import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super().__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input))
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2, 2))
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2))
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2)
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4, 1)
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4))
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5))
        # Gaussian layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6)
        return output


net = Net()
print(net)



Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
# view learnable parameters

params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [ ]:
# forward propagate with a random 32 by 32 image
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0443, -0.1379, -0.0953,  0.0236, -0.0480, -0.0589,  0.1329, -0.1700,
         -0.1390,  0.0408]], grad_fn=<AddmmBackward0>)


In [ ]:
# zero out gradient buffers with random gradients ? 
net.zero_grad()
out.backward(torch.randn(1, 10))